CPace sage notebook for generating test vectors for the CPace cipher suites.

The notebook is organized in a series of several subcells.
1.) X25519 definitions and basic string->integer->fieldElement->string conversions back and forth.
2.) Definitions of the Elligator2 primitive straight-line code from the Elligator paper, checked also against the code from the hash_to_curve team.
3.) Test vector generation for X25519 and Elligator2 as ported to sage from the ANSI-C code from the
    Endress+Hauser crypto library.
4.) Implementation and test vector generation for the CPace-specific constructions for generator
    calculation and ISK determination.
5.) Implementation and test vector generation for the AuCPace-specific constructions for generator
    calculation and SK determination.
    
The cells should best be calculated in sequence so that all definitions are available.
Tested with SageMath 9.0 on ubuntu linux.

In [41]:
# 1.) X25519 definitions and tool definitions
   
def ByteArrayToInteger(k,numBytes=32):
    return sum((k[i] << (8 * i)) for i in range(len(k)))

def IntegerToByteArray(k,numBytes = 32):
    result = bytearray(numBytes);
    for i in range(numBytes):
        result[i] = (k >> (8 * i)) & 0xff;
    return result

def IntegerToLEPrintString(u,numBytes=32):
    u = Integer(u)
    res = ""
    ctr = 0
    while ((u != 0) | (numBytes > 0)):
        byte =  u % 256
        res += ("%02x" % byte)
        u = (u - byte) >> 8
        numBytes = numBytes - 1
        ctr = ctr + 1
        if (ctr % 32) == 0:
            res += "\n"
    return res

def ByteArrayToCInitializer(k, name):
    values = [b for b in k]
    result = "const uint8_t " + name +"[] = {\n "
    for x in values:
        result += hex(x) +","
    result += "\n};"
    return result

def ByteArrayToLEPrintString(k):
    bytes = [(b) for b in k]
    res = ""
    ctr = 0
    for x in bytes:
        res += ("%02x" %x)
        ctr = ctr + 1
        if (ctr % 32) == 0:
            res += "\n"        
    return res

def tv_output_byte_array(data, test_vector_name = "", line_prefix = "  ", max_len = 70):
    string = ByteArrayToLEPrintString(data)
    print (test_vector_name + ":",end="")
    chars_per_line = max_len - len(line_prefix)
    while True:
        print ("\n" + line_prefix + string[0:chars_per_line],end="")
        string = string[chars_per_line:-1]
        if len(string) == 0:
            return

In [13]:
########## X25519 ##################

def clampScalar25519(k):
    r = bytearray(k)
    r[0] &= 248
    r[31] &= 127
    r[31] |= 64
    return r

def clampScalarForInversion25519(k):
    r = bytearray(k)
    r[0] &= 248
    return r

# all inputs to be given as byte array.
def Inverse_X25519(scalar,basepoint):
    OrderPrimeSubgroup = 2^252 + 27742317777372353535851937790883648493
    SF = GF(OrderPrimeSubgroup)
    coFactor = SF(8)
    scalarClamped = clampScalar25519(scalar)
    inverse_scalar = 1 /  (SF(ByteArrayToInteger(scalarClamped)) * coFactor)
    inverse_scalar_int = Integer(inverse_scalar) * 8
    inverse_scalar = IntegerToByteArray(inverse_scalar_int)
    return X25519(basepoint,inverse_scalar,withClamping=0)

def X25519(scalar, basepoint, withClamping=1,warnForPointOnTwist = 1):
    prime = 2^255 - 19
    F = GF(prime)
    A = F(486662)
    nonsquare = F(2)   
    E = EllipticCurve(F, [0, A , 0, 1 , 0])
    Twist = EllipticCurve(F, [0, A * nonsquare, 0, 1 * nonsquare^2, 0])

    u = F(ByteArrayToInteger(basepoint))
    if (withClamping == 1):
        clampedScalar = ByteArrayToInteger(clampScalar25519(scalar))
    else:    
        clampedScalar = ByteArrayToInteger(clampScalarForInversion25519(scalar))

        
    d = 1
    v2 = u^3 + A*u^2 + u
    if not v2.is_square():
        if (warnForPointOnTwist):
            print("Input point is on the twist! "),
        E = Twist
        d = nonsquare
        u = d * u
        v2 = u^3 + A*u^2 * nonsquare + u * nonsquare^2
    v = v2.sqrt()
    
    point = E(u, v)
    (resultPoint_u, resultPoint_v, result_Point_z) = point * clampedScalar
    resultCoordinate = resultPoint_u / d
    
    return IntegerToByteArray(Integer(resultCoordinate))

In [14]:
# 2.) Definitions for the X25519 test cases

class X25519_testCase:
    def __init__(self,u_in, s_in, u_out):
        self.u_in = u_in
        self.s_in = s_in
        self.u_out = u_out

    def runTest(self):
        us = IntegerToByteArray(self.u_in)
        ss = IntegerToByteArray(self.s_in)
        r  = IntegerToByteArray(self.u_out)
        u = X25519(ss,us)
        if (u != r):
            print ("Fail")
            print ("Input u :\n0x%032x\n" % self.u_in)
            print ("Input s :\n0x%032x\n" % self.s_in)
            print ("Correct Result :\n0x%032x\n" % self.u_out)
            print ("Actual Result :\n0x%032x\n" % ByteArrayToInteger(u))
            return False
        print ("Pass")
        return True
    
    def docOutput(self):
        print ("Test case for X25519:")
        print ("u:"),
        print (IntegerToLEPrintString(self.u_in))
        print ("s:"),
        print (IntegerToLEPrintString(self.s_in))
        print ("r:"),
        print (IntegerToLEPrintString(self.u_out))
        

testCases = []

tv = \
    X25519_testCase(0x4c1cabd0a603a9103b35b326ec2466727c5fb124a4c19435db3030586768dbe6,\
                    0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                    0x5285a2775507b454f7711c4903cfec324f088df24dea948e90c6e99d3755dac3)
testCases.append(tv)


tv = X25519_testCase(0x13a415c749d54cfc3e3cc06f10e7db312cae38059d95b7f4d3116878120f21e5,\
                     0xdba18799e16a42cd401eae021641bc1f56a7d959126d25a3c67b4d1d4e9664b,\
                    0x5779ac7a64f7f8e652a19f79685a598bf873b8b45ce4ad7a7d90e87694decb95)
testCases.append(tv)

tv = X25519_testCase(0,\
                     0xc49a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346a5,\
                     0)
testCases.append(tv)
    
weakp = []
weakp.append(0)
weakp.append(1)
weakp.append(325606250916557431795983626356110631294008115727848805560023387167927233504) #(which has order 8)
weakp.append(39382357235489614581723060781553021112529911719440698176882885853963445705823) #(which also has order 8)
weakp.append(2^255 - 19 - 1)
weakp.append(2^255 - 19)
weakp.append(2^255 - 19 + 1)
weakp.append(2^255 - 19 + 325606250916557431795983626356110631294008115727848805560023387167927233504)
weakp.append(2^255 - 19 + 39382357235489614581723060781553021112529911719440698176882885853963445705823)
weakp.append(2 * (2^255 - 19) - 1)
weakp.append(2 * (2^255 - 19))
weakp.append(2 * (2^255 - 19) + 1)

s_in = 0xff9a44ba44226a50185afcc10a4c1462dd5e46824b15163b9d7c52f06be346af;
for x in weakp:
    tv = X25519_testCase (x,s_in,0)
    testCases.append(tv)


for x in testCases:
    x.runTest()

for x in testCases:
    x.docOutput()


Pass
Input point is on the twist! 
Pass
Pass
Pass
Pass
Pass
Pass
Input point is on the twist! 
Pass
Pass
Pass
Pass
Pass
Input point is on the twist! 
Pass
Pass
Pass
Test case for X25519:
u:
e6db6867583030db3594c1a424b15f7c726624ec26b3353b10a903a6d0ab1c4c

s:
a546e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449ac4

r:
c3da55379de9c6908e94ea4df28d084f32eccf03491c71f754b4075577a28552

Test case for X25519:
u:
e5210f12786811d3f4b7959d0538ae2c31dbe7106fc03c3efc4cd549c715a413

s:
4b66e9d4d1b4673c5ad22691957d6af5c11b6421e0ea01d42ca4169e7918ba0d

r:
95cbde9476e8907d7aade45cb4b873f88b595a68799fa152e6f8f7647aac7957

Test case for X25519:
u:
0000000000000000000000000000000000000000000000000000000000000000

s:
a546e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a2244ba449ac4

r:
0000000000000000000000000000000000000000000000000000000000000000

Test case for X25519:
u:
0000000000000000000000000000000000000000000000000000000000000000

s:
af46e36bf0527c9d3b16154b82465edd62144c0ac1fc5a18506a22

In [15]:
print ("Points that need to return neutral element when input to ")
print ("plain X25519 that also accept un-normalized inputs with")
print ("bit #255 set in the input point encoding.")

ctr=0;
for x in weakp:
    print ("u"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(x),end="");
    ctr += 1;

    
print ("\nResults for X25519 implementations not clearing bit #255:")
print ("s =", IntegerToLEPrintString(s_in),end="");
print ("rN = X25519(uX, s);")
ctr=0;
for x in weakp:
    r = X25519(IntegerToByteArray(s_in), IntegerToByteArray(x),warnForPointOnTwist=0);
    r = ByteArrayToInteger(r)
    print ("r"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(r),end="");
    ctr += 1;


print ("\nResults for X25519 implementations that clear bit #255:")
print ("s =", IntegerToLEPrintString(s_in),end="");
print ("qN = X25519(uX & ((1 << 255) - 1),s);")
ctr=0;
for x in weakp:
    q = X25519(IntegerToByteArray(s_in), IntegerToByteArray(x & ((1<<255)-1)),warnForPointOnTwist=0);
    q = ByteArrayToInteger(q)
    print ("q"+'{:01x}'.format(ctr)+":",IntegerToLEPrintString(q),end="");
    ctr += 1;


Points that need to return neutral element when input to 
plain X25519 that also accept un-normalized inputs with
bit #255 set in the input point encoding.
u0: 0000000000000000000000000000000000000000000000000000000000000000
u1: 0100000000000000000000000000000000000000000000000000000000000000
u2: e0eb7a7c3b41b8ae1656e3faf19fc46ada098deb9c32b1fd866205165f49b800
u3: 5f9c95bca3508c24b1d0b1559c83ef5b04445cc4581c8e86d8224eddd09f1157
u4: ecffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u5: edffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u6: eeffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff7f
u7: cdeb7a7c3b41b8ae1656e3faf19fc46ada098deb9c32b1fd866205165f49b880
u8: 4c9c95bca3508c24b1d0b1559c83ef5b04445cc4581c8e86d8224eddd09f11d7
u9: d9ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
ua: daffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
ub: dbffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff

Results for X25

In [23]:
# 4.) Definitions for the CPace string handling primitives

def prepend_length_to_bytes(data):
    length = len(data);
    length_as_utf8_string = chr(length).encode('utf-8') # Will be a single byte in almost all real-world cases.
    return length_as_utf8_string + data 

def prefix_free_cat(*args):
    result = b""
    for arg in args:
        result += prepend_length_to_bytes(arg)
    return result

def zero_bytes(length):
    result = b"\0" * length
    return result

def generator_string(PRS,DSI,CI,sid,s_in_bytes):
    """
    Concat all input fields with prepended length information.
    Add zero padding in the first hash block between PRS and DSI.
    """
    len_zpad = max(0,s_in_bytes - 1 - len(prepend_length_to_bytes(PRS)))
    return (prefix_free_cat(PRS, zero_bytes(len_zpad), DSI, CI, sid), len_zpad)

def random_bytes(length):
    values = [randint(0, 255) for i in range(length)]
    result = b""
    for v in values:
        result += v.to_bytes(1, 'little')
    return result


In [24]:
# Definitions for the hash primitives.
import hashlib 

class H_SHA512:
    def __init__(self):
        self.b_in_bytes = 64
        self.bmax_in_bytes = 64
        self.s_in_bytes = 128
        self.name = "SHA512"
        
    def hash(self,input_str, l = 64):
        m = hashlib.sha512(input_str)
        digest = m.digest()
        return digest[0:l]

class H_SHA256:
    def __init__(self):
        self.b_in_bytes = 32
        self.bmax_in_bytes = 32
        self.s_in_bytes = 64
        self.name = "SHA256"
        
    def hash(self,input_str, l = 32):
        m = hashlib.sha256(input_str)
        digest = m.digest()
        return digest[0:l]

class H_SHAKE256:
    def __init__(self):
        self.b_in_bytes = 64
        self.bmax_in_bytes = 2^128
        self.s_in_bytes = 136
        self.name = "SHAKE256"
        
    def hash(self,input_str, l = 64):
        m = hashlib.shake_256(input_str)
        digest = m.digest(l) # Note: hashlib.shake_256 seems to be buggy in some Sage environments :-(
        return digest


In [43]:
class G_Montgomery:
    """Here we have common definitions for the X448 and X25519"""
    def sample_scalar(self):
        return random_bytes(self.field_size_bytes)

    def decodeLittleEndian(self, b):
        bits = self.field_size_bits
        num_bytes = floor((bits+7)/8)
        return sum([b[i] << 8*i for i in range(num_bytes)])

    def decodeUCoordinate(self, u):        
        u_list = [b for b in u]
        # Ignore any unused bits.
        if self.field_size_bits % 8:
            u_list[-1] &= (1<<(self.field_size_bits%8))-1
        return self.decodeLittleEndian(u_list)

    def encodeUCoordinate(self,u):
        u = u % self.q
        return IntegerToByteArray(u,self.field_size_bytes)
    
    def find_z_ell2(self,F):
        """ Argument: F, a field object, e.g., F = GF(2^255 - 19) """
        ctr = F.gen()
        while True:
            for Z_cand in (F(ctr), F(-ctr)):
                # Z must be a non-square in F.
                if is_square(Z_cand):
                    continue
                return Z_cand
            ctr += 1
        
    def elligator2(self,r):
        q = self.q
        Fq = GF(q)
        A = Fq(self.A)
        B = Fq(1)
    
        # calculate the appropriate non-square as specified in the hash2curve draft.
        u = Fq(self.find_z_ell2(Fq))
        powerForChi = floor((q-1)/2)
    
        v = - A / (1 + u * r^2)
        epsilon = (v^3 + A * v^2 + B * v)^powerForChi
        x = epsilon * v - (1 - epsilon) * A/2
        return self.encodeUCoordinate(Integer(x))

    def calculate_generator(self, H, PRS, CI, sid, print_test_vector_info = False):
        (gen_string, len_zpad) = generator_string(PRS, G.DSI,CI,sid,H.s_in_bytes)
        string_hash = H.hash(gen_string, self.field_size_bytes)
        u = self.decodeUCoordinate(string_hash)
        result = self.elligator2(u)
        if print_test_vector_info:
            print ("####  Test vectors for calculate_generator ####")
            print ("Inputs")
            print ("  H   =", H.name, "with input block size", H.s_in_bytes, "bytes.")
            print ("  PRS =", PRS, "; ZPAD length:", len_zpad,"; DSI =", G.DSI, "; CI =", CI, "")
            print ("  sid =", ByteArrayToLEPrintString(sid))
            print ("Outputs")
            tv_output_byte_array(string_hash, test_vector_name = "  hash generator string", 
                                 line_prefix = "    ", max_len = 70)
            tv_output_byte_array(IntegerToByteArray(u), test_vector_name = "  after decoding to coordinate", 
                                 line_prefix = "    ", max_len = 70)
            tv_output_byte_array(result, test_vector_name = "  generator g", 
                                 line_prefix = "    ", max_len = 70)
        return result

    
class G_X25519(G_Montgomery):
    def __init__(self):
        self.I = zero_bytes(32)
        self.field_size_bytes = 32
        self.field_size_bits = 255
        self.DSI = b"CPace255"
        
        # curve definitions
        self.q = 2^255 - 19
        self.A = 486662
        
    def scalar_mult(self,scalar,point):
        return X25519(scalar,point)

    def scalar_mult_vfy(self,scalar,point):
        return X25519(scalar,point)

    
class G_X448(G_Montgomery):
    def __init__(self):
        self.I = zero_bytes(56)
        self.field_size_bytes = 56
        self.field_size_bits = 448
        self.DSI = b"CPace448"
        
        # curve definitions
        self.q = 2^448 - 2^224 - 1
        self.A = 156326
        
    def scalar_mult(self,scalar,point):
        return X448(scalar,point) # yet no definition for X448 in this script file

    def scalar_mult_vfy(self,scalar,point):
        return X448(scalar,point) # yet no definition for X448 in this script file

#H = H_SHA512()
#G = G_X25519()
#
#G.calculate_generator(H,b"PRS",b"CI",b"sid", True)

####  Test vectors for calculate_generator ####
Inputs
  H   = SHA512 with input block size 128 bytes.
  PRS = b'PRS' ; ZPAD length: 123 ; DSI = b'CPace255' ; CI = b'CI' 
  sid = 736964
Outputs
  hash generator string:
    74d79820a64e37669ac46f858efff81e5009a4006ff0d65f7ca7f48f805bd4ec
  after decoding to coordinate:
    74d79820a64e37669ac46f858efff81e5009a4006ff0d65f7ca7f48f805bd46c
  generator g:
    0bbd8ecb04c8c21dde83ed128d103175c6f9503b48b7bd5681ff380216b90702


bytearray(b'\x0b\xbd\x8e\xcb\x04\xc8\xc2\x1d\xde\x83\xed\x12\x8d\x101u\xc6\xf9P;H\xb7\xbdV\x81\xff8\x02\x16\xb9\x07\x02')

In [21]:
# 4.) Definitions of the Elligator2 test cases
#
# Elligator 2 test cases
#
#
# Testvector from the NaCl M0 testsuite from E+H
#
G = G_X25519()

etc1_in =  0x00c84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
etc1_out = 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666

etc2_in =  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
etc2_out = 0x08a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d

ourResult1 = ByteArrayToInteger(G.elligator2(etc1_in))
ourResult2 = ByteArrayToInteger(G.elligator2(etc2_in))

if (ourResult1 != etc1_out):
    print ("Elligator test case #1 failed.")
else:
    print ("Elligator test case #1 pass.")

if (ourResult2 != etc2_out):
    print ("Elligator test case #2 failed.")
else:
    print ("Elligator test case #2 pass.")
    
print ("Elligator test case #1:")
print ("In:  0x%x" % etc1_in)
print ("Out: 0x%x" % etc1_out)

print ("Elligator test case #1:")
print ("In:  0x%x" % etc2_in)
print ("Out: 0x%x" % etc2_out)


Elligator test case #1 pass.
Elligator test case #2 pass.
Elligator test case #1:
In:  0xc84eddfa9bcd7973d6021153cd965a8a2fd749135834eaaeb093d2469a14bc
Out: 0x67d305efdb0a7c7f24ce1655ecc103126004ff23d32bfc032428cd75758fb666
Elligator test case #1:
In:  0x7563f23b0c0aa7bc27b2961a4711ba842ba303c57a9534164bf8d3b5d455cf89
Out: 0x8a3bb40e5b594b192d0ee87b663d24e1bc76d2d41c9031962a7ec6cc863b11d


In [ ]:

def CPace_ISK(sid,K,Ya,Yb,doPrint = 1):
    DSI2 = b"CPace25519-2"
    m = hashlib.sha512(DSI2)
    m.update(sid) 
    m.update(K)
    m.update(Ya)
    m.update(Yb)
    ISK = m.digest()
    if doPrint:
        print ("DSI2 = %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI2),DSI2, len(DSI2)))
        print ("sid  = %s string of len(%i)" % (ByteArrayToLEPrintString(sid), len(sid)))
        print ("K    = %s"\
               "       string of len(%i)" % (ByteArrayToLEPrintString(K), len(K)))
        print ("Ya   = %s"\
               "       string of len(%i)" % (ByteArrayToLEPrintString(Ya), len(Ya)))
        print ("Yb   = %s"\
               "       string of len(%i)" % (ByteArrayToLEPrintString(Yb), len(Yb)))
        print ("ISK  = SHA512(DSI2 || sid || K || Ya || Yb)\n" \
               "     = %s" \
               "       string of len(%i)" % (ByteArrayToLEPrintString(ISK), len(ISK)))
    return ISK



sid = hashlib.sha512(b"sid").digest()
sid = sid [:16]
PRS = prepend_length_to_bytes(b"password")
CI = (prepend_length_to_bytes(b"Ainitiator") 
     + prepend_length_to_bytes(b"Bresponder") 
     + prepend_length_to_bytes(b"AD"))

print ("Derivation of the secret generator G\n")
G = map_to_group_mod_neg_CPace25519(sid,PRS,CI,1)

ya = hashlib.sha512(b"ya").digest()
ya = bytearray(ya [:32])
yb = hashlib.sha512(b"yb").digest()
yb = bytearray(yb [:32])

Ya = X25519(G,ya)
Yb = X25519(G,yb)

K1 = X25519(Yb,ya)
K2 = X25519(Ya,yb)

if (K1 != K2):
    print ("Diffie-Hellman did fail!")
else:
    print ("Diffie-Hellman pass.")
    K = K1
    
print ("\nSecret scalar ya=SHA512(\'ya\'), bytes 0...31, as integer:\n  0x%x" % ByteArrayToInteger(ya))
print ("Secret scalar yb=SHA512(\'yb\'), bytes 0...31, as integer:\n  0x%x" % ByteArrayToInteger(yb))

print ("Public point Ya as integer:\n  0x%x" % ByteArrayToInteger(Ya))
print ("Public point Yb as integer:\n  0x%x" % ByteArrayToInteger(Yb))

print ("DH point K as integer:\n  0x%x" % ByteArrayToInteger(K1))

print ("\nDerivation of the intermediate session key ISK:\n")
ISK = KDF_CPace25519(sid,K,Ya,Yb,1)

In [ ]:
# 5.) Test the inverse X25519 operation for strong AuCPace.

# generate deterministic random numbers based on SHA512 and different strings.
# Generate points and scalars from such random inputs.
# Check that the inverse operation is actually inverting X25519.
import hashlib

def map_to_group_mod_neg_StrongAuCPace25519(username, password, doPrint = 1, DSI = b"AuCPace25519"):
    F = GF(2^255 - 19)
    m = hashlib.sha512()
    H_block_SHA512 = 128
    ZPAD_len = max(0,H_block_SHA512 - len(DSI) - len(password))
    ZPAD = ZPAD_len * b"\0"
        
    if doPrint:
        print ("DSI5 = %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI),DSI, len(DSI)))
        print ("pw   = %s (%s) string of len(%i)" % (ByteArrayToLEPrintString(password), password, len(password)))
        print ("ZPAD = %i zero bytes (before mixing in adversary controlled variable data)" % (len(ZPAD)))
        print ("name = %s (%s) string of len(%i)" % (ByteArrayToLEPrintString(username), username, len(username)))
    m.update(DSI)
    m.update(password)
    m.update(ZPAD)
    m.update(username) 
    u = ByteArrayToInteger(bytearray(m.digest()),64)
    
    if doPrint:
        print ("u = SHA512(DSI||password||ZPAD||username) as 512 bit little-endian int:\n  0x%032x << 256\n+ 0x%32x" \
               % (Integer(u) >> 256, Integer(u) & (2^256 - 1)))
    u = F(u)
    Z = elligator2_curve25519(IntegerToByteArray(Integer(u)))
    if doPrint:
        print ("u as reduced base field element coordinate:\n  0x%x" % Integer(u))
        print ("Elligator2 output Z as base field element coordinate:\n  0x%x" % ByteArrayToInteger(Z))
    return Z

def MAC_SK_AuCPace25519(ISK,doPrint = 1):
    DSI3 = b"AuCPace25-Ta"
    DSI4 = b"AuCPace25-Tb"
    DSI5 = b"AuCPace25519"
    m = hashlib.sha512(DSI3)
    m.update(ISK) 
    Ta = m.digest()
    Ta = Ta[:16]

    m = hashlib.sha512(DSI4)
    m.update(ISK) 
    Tb = m.digest()
    Tb = Tb[:16]

    m = hashlib.sha512(DSI5)
    m.update(ISK) 
    SK = m.digest()
    if doPrint:
        print ("DSI3 = %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI3),DSI3, len(DSI3)))
        print ("DSI4 = %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI4),DSI4, len(DSI4)))
        print ("DSI5 = %s string (%s) of len(%i)" % (ByteArrayToLEPrintString(DSI5),DSI5, len(DSI5)))
        print ("ISK  = %s string of len(%i)" % (ByteArrayToLEPrintString(ISK), len(ISK)))
        print ("Ta  = %s string of len(%i)" % (ByteArrayToLEPrintString(Ta), len(Ta)))
        print ("Tb  = %s string of len(%i)" % (ByteArrayToLEPrintString(Tb), len(Tb)))
        print ("SK  = %s string of len(%i)" % (ByteArrayToLEPrintString(SK), len(SK)))
    return (Ta,Tb,SK)


# First add corner case such that the inverse scalar of an X25519 input 
# has a value larger than  2^252 mod
OrderPrimeSubgroup = 2^252 + 27742317777372353535851937790883648493
SF = GF(OrderPrimeSubgroup)
coFactor = SF(8)

inverseX25519_scalar = 2^252 + 1
while True:
    X25519_scalar = 8 * Integer(SF(inverseX25519_scalar * 8))
    if (X25519_scalar & (2^254)):
        break;
    inverseX25519_scalar += 8

X25519_scalar = IntegerToByteArray(X25519_scalar)
inverseX25519_scalar = IntegerToByteArray(inverseX25519_scalar)

print ("Corner test case where the inverse of the X25519 scalar is ")
print ("larger than 2^252 modulo the prime order subgroup:")

B = IntegerToByteArray(9) # base point coordinate of Curve25519

inStr = "x"
s = hashlib.sha512(inStr.encode()).digest()
s = bytearray(s [:32])
T = X25519(B,s)
print ("T  : 0x%x" % ByteArrayToInteger(T))

print ("r  : 0x%x" % ByteArrayToInteger(X25519_scalar))
print ("cr : 0x%x" % ByteArrayToInteger(clampScalar25519(X25519_scalar)))
clampScalar25519
print ("1/r: 0x%x" % ByteArrayToInteger(inverseX25519_scalar))

U = X25519(T,X25519_scalar)
print ("U  : 0x%x" % ByteArrayToInteger(U))
IU = Inverse_X25519(U,X25519_scalar)
print ("IU : 0x%x" % ByteArrayToInteger(IU))
error = ByteArrayToInteger(IU) - ByteArrayToInteger(T)
if (error):
    print ("Error : 0x%x" % (error))

print ("\n\n\nCorner case of an input point G to inverseX25519 that is")
print ("not on the prime-order subgroup of Curve25519:\n")

# Determine one by some special password and Elligator2. 
# Note that most passwords won't generate points on the prime
# order subgroup.
ctr = 0
while True:
    passwordString = b"%i" % ctr
    
    G_test = map_to_group_mod_neg_StrongAuCPace25519(b"username",passwordString,doPrint = 0)
    U_test = X25519(G_test,X25519_scalar)
    IG_test = Inverse_X25519(G_test,X25519_scalar)
    IU_test = Inverse_X25519(U_test,X25519_scalar)
    error = ByteArrayToInteger(IU_test) - ByteArrayToInteger(G_test)
    if (error != 0):
        break
    ctr += 1

Z_test = map_to_group_mod_neg_StrongAuCPace25519(b"username",passwordString,doPrint = 1)

U_test = X25519(G_test,X25519_scalar)
IU_test = Inverse_X25519(U_test,X25519_scalar)
IZ_test = Inverse_X25519(Z_test,X25519_scalar)
XIZ_test = X25519(IZ_test,X25519_scalar)

print ("\nU   = X25519(Z,r)")
print ("IU  = inverse_X25519(U,r)")
print ("IZ  = inverse_X25519(Z,r)")
print ("ZIG = X25519(inverse_X25519(Z,r),r)\n")

print ("Z  : 0x%x" % ByteArrayToInteger(G_test))
print ("r  : 0x%x" % ByteArrayToInteger(X25519_scalar))
print ("U  : 0x%x" % ByteArrayToInteger(U_test))
print ("IU : 0x%x" % ByteArrayToInteger(IU_test))
print ("IZ : 0x%x" % ByteArrayToInteger(IG_test))
#print ("XIZ: 0x%x" % ByteArrayToInteger(XIG_test))

print ("Note that both times the inverse operations don't come back to Z!")


print ("\n\n\nRandom test vectors:")
        
inStr = "a"
for m in range(10):
    s = hashlib.sha512(inStr.encode()).digest()
    inStr += "x"
    s = bytearray(s [:32])

    T = X25519(B,s)
    print ("T : 0x%x" % ByteArrayToInteger(T))

    inStr += "x"

    r = hashlib.sha512(inStr.encode()).digest()
    r = bytearray(r [:32])
    print ("r : 0x%x" % ByteArrayToInteger(r))

    U = X25519(T,r)
    print ("U : 0x%x" % ByteArrayToInteger(U))

    IU = Inverse_X25519(U,r)
    #integerToLittleEndianString(Inverse_X25519(U,r))
    print ("IU : 0x%x" % ByteArrayToInteger(IU))
    error = ByteArrayToInteger(IU) - ByteArrayToInteger(T)
    if (error):
        print ("Error : 0x%x" % (error))

print ("End")

In [ ]:
#6.) Test vectors for AuCPace


name = b"username"
password = b"password"

q = hashlib.sha512(b"q").digest()
q = bytearray(q [:32])

r = hashlib.sha512(b"r").digest()
r = bytearray(r [:32])

Z = map_to_group_mod_neg_StrongAuCPace25519(name,password)

print ("\n\n\n")
print ("Z for username, password:\n0x%x" % ByteArrayToInteger(Z))

print ("q:\n0x%x" % ByteArrayToInteger(q))


U = X25519(Z,r)
print ("R:\n0x%x" % ByteArrayToInteger(r))

print ("U:\n0x%x" % ByteArrayToInteger(U))

UQ = X25519(U,q)

print ("UQ:\n0x%x" % ByteArrayToInteger(UQ))

ZQ = X25519(Z,q)

print ("ZQ directly calculated:\n0x%x" % ByteArrayToInteger(ZQ))

ZQ2 = Inverse_X25519(UQ,r)
print ("ZQ as calculated from UQ:\n0x%x" % ByteArrayToInteger(ZQ2))

def ByteArrayToString(k):
    chars = [chr(b) for b in k]
    result = ""
    for x in chars:
        result += x
    return result

w = hashlib.scrypt(password + name, salt = ZQ, n = int(1 << 15),r = int(8),\
                    p = int(1),dklen=int(32),maxmem=int(2^30))

w = bytearray(w)
x = hashlib.sha512(b"x").digest()
x = bytearray(x[:32])


W = X25519(B,w)
X = X25519(B,x)


XW = X25519(W,x)
WX = X25519(X,w)

print("Password: \'%s\', length %i" % (password,len(password)))
print("User Name: \'%s\', length %i" % (name,len(name)))
print("Concatenated input to scrypt: %s \'%s\', length %i" % (ByteArrayToLEPrintString(password+name),\
                                                              (password+name), \
                                                             len(password+name)))
print("Salt value Z^q:\n0x%x" % ByteArrayToInteger(ZQ))
print("Password hash w from scrypt with N=1<<15, r=8, p=1\n0x%x" % ByteArrayToInteger(w))

print("Password verifier W = X25519(B,w)\n0x%x" % ByteArrayToInteger(X25519(B,w)))
print("Server private key x = \n0x%x" % ByteArrayToInteger(x))
print("Server Public key X = X25519(B,x)\n0x%x" % ByteArrayToInteger(X25519(X,w)))

print("Shared secret XW = X25519(W,x)\n0x%x" % ByteArrayToInteger(X25519(W,x)))
print("Shared secret WX = X25519(X,w)\n0x%x" % ByteArrayToInteger(X25519(X,w)))


ISK_test2 = hashlib.sha512(b"ISK").digest()

(Ta, Tb, SK) = MAC_SK_AuCPace25519(ISK_test2)

In [ ]:
# Test vectors for the C Code, Elligator2

print (ByteArrayToCInitializer(IntegerToByteArray(etc1_in), "EllTestCase1_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc1_out), "EllTestCase1_out"))

print (ByteArrayToCInitializer(IntegerToByteArray(etc2_in), "EllTestCase2_in"))
print (ByteArrayToCInitializer(IntegerToByteArray(etc2_out), "EllTestCase2_out"))

tc3_in = bytearray(hashlib.sha512(b"LongInput").digest())
print (ByteArrayToCInitializer(tc3_in, "EllTestCase3_in"))

tc3_out = elligator2_curve25519(tc3_in)
print (ByteArrayToCInitializer(tc3_out, "EllTestCase3_out"))




In [ ]:
#7.) Test vector output for ANSI-C "strong" AuCPace salt blinding.
print (ByteArrayToCInitializer(Z, "tc_Z"))
print (ByteArrayToCInitializer(r, "tc_r"))
print (ByteArrayToCInitializer(U, "tc_U"))
print (ByteArrayToCInitializer(q, "tc_q"))
print (ByteArrayToCInitializer(UQ, "tc_UQ"))
print (ByteArrayToCInitializer(ZQ, "tc_ZQ"))


In [ ]:
#8.) Test vector output for ANSI-C, CPace

print (ByteArrayToCInitializer(G, "tc_G"))
print (ByteArrayToCInitializer(ya, "tc_ya"))
print (ByteArrayToCInitializer(Ya, "tc_Ya"))
print (ByteArrayToCInitializer(yb, "tc_yb"))
print (ByteArrayToCInitializer(Yb, "tc_Yb"))
print (ByteArrayToCInitializer(K1, "tc_K"))
print (ByteArrayToCInitializer(ISK, "tc_ISK"))



In [ ]:
#9.) Test vectors for ANSI-C, augmentation layer and scrypt
print (ByteArrayToCInitializer(w, "tc_w"))
print (ByteArrayToCInitializer(X, "tc_X"))
print (ByteArrayToCInitializer(WX, "tc_WX"))
print (ByteArrayToCInitializer(ZQ, "tc_ZQ"))

In [ ]:
#10.) Test vectors for ANSI-C, SK calculation

print (ByteArrayToCInitializer(ISK_test2, "tc_ISK_test2"))
print (ByteArrayToCInitializer(Ta, "tc_Ta"))
print (ByteArrayToCInitializer(Tb, "tc_Tb"))
print (ByteArrayToCInitializer(SK, "tc_SK"))


In [ ]:
def prepend_length_to_bytes(data):
    length = len(data);
    length_as_utf8_string = chr(length).encode('utf-8') # Will be a single byte in almost all real-world cases
    return length_as_utf8_string + data 

print (prepend_length_to_bytes (b"Hallo") + b"2")

PRS = prepend_length_to_bytes(b"password")
CI = prepend_length_to_bytes(b"Ainitiator") + prepend_length_to_bytes(b"Bresponder") + prepend_length_to_bytes(b"AD")
